In [3]:
import numpy as np
import time
import math
import random
from typing import List, Tuple, Dict

def read_tsp_file(filename: str) -> List[Tuple[float, float]]:
    """TSP dosyasını okur ve koordinatları döndürür."""
    coordinates = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        num_cities = int(lines[0].strip())
        for i in range(1, num_cities + 1):
            if i < len(lines):
                values = lines[i].strip().split()
                if len(values) >= 2:
                    x = float(values[0])
                    y = float(values[1])
                    coordinates.append((x, y))
    return coordinates

def calculate_distance(point1: Tuple[float, float], point2: Tuple[float, float]) -> float:
    """İki nokta arasındaki Öklid mesafesini hesaplar."""
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def build_distance_matrix(coordinates: List[Tuple[float, float]]) -> np.ndarray:
    """Koordinatlardan uzaklık matrisini oluşturur."""
    n = len(coordinates)
    distance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):  # Simetrik matris, sadece üst üçgeni hesapla
            dist = calculate_distance(coordinates[i], coordinates[j])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # Simetrik olduğu için
    
    return distance_matrix

def nearest_neighbor_tsp(distance_matrix: np.ndarray, start_city: int = 0) -> Tuple[List[int], float]:
    """Nearest Neighbor algoritması ile TSP çözümü."""
    n = distance_matrix.shape[0]
    visited = [False] * n
    path = [start_city]
    visited[start_city] = True
    total_distance = 0.0
    
    for _ in range(n - 1):
        last_city = path[-1]
        min_distance = float('inf')
        nearest_city = -1
        
        for j in range(n):
            if not visited[j] and distance_matrix[last_city, j] < min_distance:
                min_distance = distance_matrix[last_city, j]
                nearest_city = j
        
        if nearest_city != -1:
            visited[nearest_city] = True
            path.append(nearest_city)
            total_distance += min_distance
    
    # Son şehirden başlangıç şehrine geri dön
    total_distance += distance_matrix[path[-1], path[0]]
    path.append(path[0])  # Çevrimi tamamla
    
    return path, total_distance

def calculate_route_length(route: List[int], distance_matrix: np.ndarray) -> float:
    """Verilen rota için toplam mesafeyi hesaplar."""
    return sum(distance_matrix[route[i], route[i+1]] for i in range(len(route)-1))

def two_opt_swap(route: List[int], i: int, k: int) -> List[int]:
    """i ve k arasındaki yol segmentini ters çevirir."""
    new_route = route[:i]
    new_route.extend(reversed(route[i:k+1]))
    new_route.extend(route[k+1:])
    return new_route

def two_opt_improvement(distance_matrix: np.ndarray, initial_path: List[int], 
                        max_iterations: int = 1000, improvement_threshold: float = 0.0001) -> Tuple[List[int], float]:
    """2-opt algoritması ile mevcut çözümü iyileştirir."""
    # Başlangıç düğümüne dönüşü geçici olarak kaldır
    if initial_path[0] == initial_path[-1]:
        path = initial_path[:-1].copy()
    else:
        path = initial_path.copy()
    
    n = len(path)
    
    # İlk başta path'i en iyi çözüm olarak kabul et
    best_path = path.copy()
    
    # İlk çözümün mesafesini hesapla
    current_route = path + [path[0]]  # Çevrimi tamamla
    current_distance = calculate_route_length(current_route, distance_matrix)
    best_distance = current_distance
    
    improvement = True
    iteration = 0
    
    while improvement and iteration < max_iterations:
        improvement = False
        iteration += 1
        
        # 318 düğüm için tam 2-opt arama yerine daha verimli bir yaklaşım
        # Her iterasyonda bazı rastgele seçilmiş kenar çiftlerini deneyelim
        edge_pairs = []
        for i in range(n - 2):
            for j in range(i + 2, n):
                if not (i == 0 and j == n - 1):  # Son ve ilk düğüm arasındaki kenarı dışarıda bırak
                    edge_pairs.append((i, j))
        
        # Kenar çiftlerini karıştır
        random.shuffle(edge_pairs)
        
        # İlk 500 çifti dene
        for i, j in edge_pairs[:min(500, len(edge_pairs))]:
            # Mevcut kenarlar: (path[i], path[i+1]) ve (path[j], path[j+1 % n])
            # Yeni kenarlar: (path[i], path[j]) ve (path[i+1], path[j+1 % n])
            
            # 2-opt değişimi için potansiyel kazanç hesapla
            current_cost = distance_matrix[path[i], path[i+1]] + distance_matrix[path[j], path[(j+1) % n]]
            new_cost = distance_matrix[path[i], path[j]] + distance_matrix[path[i+1], path[(j+1) % n]]
            
            if new_cost < current_cost:
                # Değişimi yap
                path = two_opt_swap(path, i+1, j)
                improvement = True
                break  # İyileştirme bulduğumuzda bu iterasyonu sonlandır
        
        # Mevcut çözümün toplam mesafesini hesapla
        current_route = path + [path[0]]  # Çevrimi tamamla
        current_distance = calculate_route_length(current_route, distance_matrix)
        
        # Eğer daha iyi bir çözüm bulunduysa güncelle
        if current_distance < best_distance:
            best_distance = current_distance
            best_path = path.copy()
            print(f"İterasyon {iteration}: Yeni en iyi mesafe = {best_distance:.2f}")
        
        # İyileşme çok azsa durabilirsiniz
        if improvement and abs(best_distance - current_distance) / best_distance < improvement_threshold:
            print(f"Yeterli iyileşme yok, durduruluyor...")
            break
    
    # Başlangıç düğümünü sona ekleyerek çevrimi tamamla
    best_path.append(best_path[0])
    
    return best_path, best_distance

def simulated_annealing_tsp(distance_matrix: np.ndarray, initial_solution: List[int], 
                           initial_temp: float = 100, cooling_rate: float = 0.95, 
                           min_temp: float = 1, max_iterations: int = 50) -> Tuple[List[int], float]:
    """Tavlama Benzetimi ile TSP çözümü."""
    # Başlangıç düğümüne dönüşü geçici olarak kaldır
    if initial_solution[0] == initial_solution[-1]:
        current_solution = initial_solution[:-1].copy()
    else:
        current_solution = initial_solution.copy()
    
    n = len(current_solution)
    best_solution = current_solution.copy()
    
    # Mevcut çözümün mesafesini hesapla
    current_route = current_solution + [current_solution[0]]  # Çevrimi tamamla
    current_distance = calculate_route_length(current_route, distance_matrix)
    best_distance = current_distance
    
    temp = initial_temp
    
    # Her sıcaklık seviyesi için iterasyon sayısı
    iterations_per_temp = n * 5
    
    print(f"Tavlama Benzetimi başlatılıyor...")
    print(f"Başlangıç mesafesi: {best_distance:.2f}")
    
    # Her sıcaklık seviyesi için ana döngü
    temp_iterations = 0
    while temp > min_temp and temp_iterations < max_iterations:
        print(f"Sıcaklık: {temp:.2f}, Mevcut mesafe: {current_distance:.2f}, En iyi mesafe: {best_distance:.2f}")
        temp_iterations += 1
        
        # Her sıcaklık seviyesinde belirli sayıda iterasyon yap
        for _ in range(iterations_per_temp):
            # Rastgele iki şehir seç (i != j)
            i, j = random.sample(range(n), 2)
            
            # İndeksleri sırala
            if i > j:
                i, j = j, i
            
            # Yeni çözüm için 2-opt değişimi yap
            new_solution = two_opt_swap(current_solution, i, j)
            
            # Çevrimi tamamlayarak yeni mesafeyi hesapla
            new_route = new_solution + [new_solution[0]]
            new_distance = calculate_route_length(new_route, distance_matrix)
            
            # Değişimin mesafe farkını hesapla
            delta = new_distance - current_distance
            
            # Eğer yeni çözüm daha iyi veya belirli bir olasılıkla kabul edilebilir
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_solution = new_solution
                current_distance = new_distance
                
                # Eğer yeni çözüm en iyisinden daha iyiyse, en iyi çözümü güncelle
                if current_distance < best_distance:
                    best_solution = current_solution.copy()
                    best_distance = current_distance
                    print(f"Yeni en iyi mesafe: {best_distance:.2f}")
        
        # Sıcaklığı azalt
        temp *= cooling_rate
    
    # Başlangıç düğümünü sona ekleyerek çevrimi tamamla
    best_solution.append(best_solution[0])
    
    return best_solution, best_distance

def print_full_path(path: List[int], distance: float):
    """Tüm tur yolunu ekrana yazdırır."""
    print("\n" + "="*60)
    print("TSP ÇÖZÜM SONUÇLARI")
    print("="*60)
    print(f"Optimal maliyet değeri: {distance:.2f}")
    print("Optimal maliyeti sağlayan path:")
    # Tüm yolu doğrudan yazdır
    path_str = ' -> '.join(map(str, path))
    print(path_str)
    print("="*60)

def solve_tsp_318(filename: str = "tsp_318.txt") -> Dict:
    """318 düğümlü TSP problemini çözer."""
    print(f"TSP Çözücü - {filename} için başlatılıyor...")
    
    start_time = time.time()
    
    # Dosyayı oku
    try:
        coordinates = read_tsp_file(filename)
        n = len(coordinates)
        print(f"Toplam {n} şehir koordinatı okundu.")
    except Exception as e:
        print(f"Dosya okuma hatası: {e}")
        return {}
    
    # Mesafe matrisini oluştur
    print("Mesafe matrisi oluşturuluyor...")
    distance_matrix = build_distance_matrix(coordinates)
    
    # Nearest Neighbor ile başlangıç çözümü
    print("Nearest Neighbor algoritması başlatılıyor...")
    nn_start_time = time.time()
    nn_path, nn_distance = nearest_neighbor_tsp(distance_matrix)
    nn_time = time.time() - nn_start_time
    
    print(f"Nearest Neighbor çözümü: mesafe = {nn_distance:.2f}, süre = {nn_time:.2f} saniye")
    
    # 2-opt ile iyileştirme
    print("2-opt iyileştirmesi başlatılıyor...")
    twoopt_start_time = time.time()
    twoopt_path, twoopt_distance = two_opt_improvement(distance_matrix, nn_path)
    twoopt_time = time.time() - twoopt_start_time
    
    print(f"2-opt iyileştirmesi: mesafe = {twoopt_distance:.2f}, iyileşme = {(nn_distance-twoopt_distance)/nn_distance*100:.2f}%, süre = {twoopt_time:.2f} saniye")
    
    # Tavlama Benzetimi ile daha fazla iyileştirme
    print("Tavlama Benzetimi ile iyileştirme başlatılıyor...")
    sa_start_time = time.time()
    try:
        sa_path, sa_distance = simulated_annealing_tsp(distance_matrix, twoopt_path, 
                                                     initial_temp=100, cooling_rate=0.95)
        sa_time = time.time() - sa_start_time
        print(f"Tavlama Benzetimi: mesafe = {sa_distance:.2f}, iyileşme = {(twoopt_distance-sa_distance)/twoopt_distance*100:.2f}%, süre = {sa_time:.2f} saniye")
    except Exception as e:
        print(f"Tavlama Benzetimi hatası: {e}")
        sa_path = twoopt_path
        sa_distance = twoopt_distance
        sa_time = 0
    
    # En iyi çözümü belirle
    if sa_distance < twoopt_distance:
        best_path = sa_path
        best_distance = sa_distance
        print("En iyi çözüm: Tavlama Benzetimi sonucu")
    else:
        best_path = twoopt_path
        best_distance = twoopt_distance
        print("En iyi çözüm: 2-opt iyileştirmesi sonucu")
    
    # Toplam süre
    total_time = time.time() - start_time
    print(f"Toplam çalışma süresi: {total_time:.2f} saniye")
    
    # Tüm yolu ekrana yazdır
    print_full_path(best_path, best_distance)
    
    return {
        "path": best_path,
        "distance": best_distance,
        "time": total_time,
        "nn_distance": nn_distance,
        "twoopt_distance": twoopt_distance,
        "sa_distance": sa_distance
    }

if __name__ == "__main__":
    solve_tsp_318()

TSP Çözücü - tsp_318.txt için başlatılıyor...
Toplam 318 şehir koordinatı okundu.
Mesafe matrisi oluşturuluyor...
Nearest Neighbor algoritması başlatılıyor...
Nearest Neighbor çözümü: mesafe = 53245.15, süre = 0.01 saniye
2-opt iyileştirmesi başlatılıyor...
2-opt iyileştirmesi: mesafe = 53245.15, iyileşme = 0.00%, süre = 0.03 saniye
Tavlama Benzetimi ile iyileştirme başlatılıyor...
Tavlama Benzetimi başlatılıyor...
Başlangıç mesafesi: 53245.15
Sıcaklık: 100.00, Mevcut mesafe: 53245.15, En iyi mesafe: 53245.15
Sıcaklık: 95.00, Mevcut mesafe: 53879.29, En iyi mesafe: 53245.15
Sıcaklık: 90.25, Mevcut mesafe: 54758.98, En iyi mesafe: 53245.15
Sıcaklık: 85.74, Mevcut mesafe: 54982.83, En iyi mesafe: 53245.15
Sıcaklık: 81.45, Mevcut mesafe: 55540.08, En iyi mesafe: 53245.15
Sıcaklık: 77.38, Mevcut mesafe: 55617.63, En iyi mesafe: 53245.15
Sıcaklık: 73.51, Mevcut mesafe: 56201.96, En iyi mesafe: 53245.15
Sıcaklık: 69.83, Mevcut mesafe: 55754.19, En iyi mesafe: 53245.15
Sıcaklık: 66.34, Mevcut